# Eat Safe, Love

## Notebook Set Up

In [27]:
# Import dependencies
from pymongo import MongoClient
# Import pprint
from pprint import pprint
import pandas as pd

In [28]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [29]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [30]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [31]:
# assign the collection to a variable
estab = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [32]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': {'$eq': 20}}

# Use count_documents to display the number of documents in the result
print(estab.count_documents(query))

# Display the first document in the results using pprint
pprint(estab.find_one(query))

0
None


In [33]:
# Convert the result to a Pandas DataFrame
results = list(estab.find(query))
hygiene = pd.DataFrame(results)

# Display the first 10 rows of the DataFrame
hygiene.head(10)

""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [34]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query ={'LocalAuthorityName': {'$regex': 'London'},
         'RatingValue': {'$gte': '4'}}

# Use count_documents to display the number of documents in the result
print(estab.count_documents(query))

# Display the first document in the results using pprint
pprint(estab.find_one(query))


0
None


In [35]:
# Convert the result to a Pandas DataFrame
results = list(estab.find(query))
London = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(London))
# Display the first 10 rows of the DataFrame
London.head(10)

Rows in DataFrame:  0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [36]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query ={'geocode.latitude': {'$gte': (latitude - degree_search)},
         'geocode.latitude': {'$lte': (latitude + degree_search)},
        'geocode.longitude': {'$gte': (longitude - degree_search)},
         'geocode.longitude': {'$lte': (longitude + degree_search)},
       'RatingValue': {'$eq': '5'}}
sort = [('scores.Hygiene', 1)]
limit = 5
results = estab.find(query).sort(sort).limit(limit)

# Pretty print the first five results
for result in results:
    pprint(result)

In [37]:
# Convert result to Pandas DataFrame
rating = pd.DataFrame(list(estab.find(query).sort(sort).limit(limit)))
rating.head()

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [38]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
match_query = {'$match': {'scores.Hygiene': {'$eq': 0}}}
# 2. Groups the matches by Local Authority
group_query = {'$group': {'_id': "$LocalAuthorityName", 'count': { '$sum': 1 }}}
# 3. Sorts the matches from highest to lowest
sort_values = {'$sort': { 'count': -1 }}
pipeline = [match_query, group_query, sort_values]
# Print the number of documents in the result
results = estab.aggregate(pipeline)
# Print the first 10 results
i=0
for result in results:
    if i in range(10):
        pprint(result)
        i = i+1

In [39]:
# Convert the result to a Pandas DataFrame
data = list(estab.aggregate(pipeline))
local = pd.DataFrame(data)
# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(local))
# Display the first 10 rows of the DataFrame
local.head(10)

Rows in DataFrame:  0


""
